# 要約データの取得

Live DoorニュースIDデータのGitHubリポジトリ<br>
https://github.com/KodairaTomonori/ThreeLineSummaryDataset

In [1]:
from pathlib import Path
import re
import time
import requests
import pandas as pd
import urllib
from bs4 import BeautifulSoup
from tqdm import tqdm

In [28]:
%load_ext autoreload
%autoreload 2
pd.set_option('max_colwidth', 1000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## パラメータの設定

In [3]:
data_dir_path = Path('data')
if not data_dir_path.exists():
    data_dir_path.mkdir(parents=True)

## ニュースのIDを取得

In [4]:
def download_data(url, data_dir_path):
    
    file_path = data_dir_path.joinpath(Path(url).name)

    data = requests.get(url).content
    with open(file_path, 'wb') as file:
        file.write(data)

In [5]:
url = 'https://raw.githubusercontent.com/KodairaTomonori/ThreeLineSummaryDataset/master/data/train.csv'
download_data(url=url, data_dir_path=data_dir_path)

In [6]:
url = 'https://raw.githubusercontent.com/KodairaTomonori/ThreeLineSummaryDataset/master/data/test.csv'
download_data(url=url, data_dir_path=data_dir_path)

### ニュースIDデータの加工

In [8]:
def anti_join(data1, data2, by):

    joined_data = data1.copy()
    target_data = data2.copy()
    target_data['flag_tmp'] = 1

    if type(by) is str:
        by = [by]

    joined_data = pd.merge(
        joined_data, target_data[by + ['flag_tmp']].drop_duplicates(),
        on=by, how='left'
    ).query('flag_tmp.isnull()', engine='python').drop(columns='flag_tmp').copy()

    return joined_data

In [17]:
columns = ['year', 'month', 'category', 'article_id', 'type_label']

articles = pd.DataFrame()
for data_name in ['train.csv', 'test.csv']:
    data = pd.read_csv(data_dir_path.joinpath(data_name))
    tmp = data.columns.tolist()
    if data_name == 'train.csv':
        data.columns = columns[:-1]
        data = pd.concat([
            data, pd.DataFrame([tmp], columns=columns[:-1])
        ], axis=0)
        data['type_label'] = None
    else:
        data.columns = columns
        data = pd.concat([
            data, pd.DataFrame([tmp], columns=columns)
        ], axis=0)
        
    articles = pd.concat([articles, data], axis=0)

articles = articles.assign(
    year=lambda x: x.year.astype(int),
    article_id=lambda x: x.article_id.map(lambda y: re.sub(r'[a-z\.]', '', str(y))).astype(int)
)

if body_data_file_path.exists():
    articles = anti_join(
        articles,
        pd.read_csv(body_data_file_path).assign(article_id=lambda x: x.article_id.astype(int)),
        by='article_id'
    )

## ニューステキストとサマリを取得

In [31]:
waiting_time = 3              # スクレイピングの間隔
n_writing_data = 10        # 取得する件数 
article_url = 'http://news.livedoor.com/article/detail/{}/'
body_data_file_path = data_dir_path.joinpath('body_data.csv')
summary_data_file_path = data_dir_path.joinpath('summary_data.csv')

In [33]:
target_articles = articles.sort_values(
    'year', ascending=False
).head(min(len(articles), n_writing_data))

### ニューステキストとサマリをスクレイピング

In [19]:
def read_url_to_soup(url):
    
    try:
        response = urllib.request.urlopen(url)
        html = response.read().decode(response.headers.get_content_charset(), errors='ignore')
        soup = BeautifulSoup(html, 'html.parser')
    except Exception:
        soup = None
    
    return soup

In [20]:
def write_data(data, file_path):
    if file_path.exists():
        data = pd.concat([data, pd.read_csv(file_path)]).assign(
            article_id=lambda x: x.article_id.astype(int)
        ).drop_duplicates()
    data.to_csv(file_path, index=False)

In [34]:
body_data = []
summary_data = []
i = 1
for article_id in tqdm(target_articles['article_id']):

    url = article_url.format(article_id)

    soup = read_url_to_soup(url)
    if soup is None or soup.find(class_='articleBody') is None or soup.find(class_='summaryList') is None:
        body_data.append((article_id, None, None))
        summary_data.append((article_id, None))
    else:

        title = soup.find(id='article-body').find('h1').text.strip()

        body = soup.find(class_='articleBody').find('span', {'itemprop': 'articleBody'}).text
        body = re.sub('\n+', '\n', body)
        body_data.append((article_id, title, body))

        summary_list = soup.find(class_='summaryList').find_all('li')
        summary_list = list(map(lambda x: x.text.strip(), summary_list))

        summary_data.extend([(article_id, summary) for summary in summary_list])
    
    if i % 50 == 0:        
        body_data = pd.DataFrame(body_data, columns=['article_id', 'title', 'text'])
        summary_data = pd.DataFrame(summary_data, columns=['article_id', 'text'])
        write_data(data=body_data, file_path=body_data_file_path)
        write_data(data=summary_data, file_path=summary_data_file_path)
        body_data = []
        summary_data = []        

    i += 1
    time.sleep(waiting_time)

if len(body_data) > 0:
    body_data = pd.DataFrame(body_data, columns=['article_id', 'title', 'text'])
    summary_data = pd.DataFrame(summary_data, columns=['article_id', 'text'])
    write_data(data=body_data, file_path=body_data_file_path)
    write_data(data=summary_data, file_path=summary_data_file_path)

100%|██████████| 10/10 [00:38<00:00,  3.81s/it]


#### 取得したデータの確認

In [29]:
data = pd.read_csv(body_data_file_path)
data.head()

,article_id,title,text
0,11574091,滝沢秀明の連ドラ主演にファンから「あの脇役に食われる！」と心配の声,\n 7月スタートの連ドラ「せいせいするほど、愛してる」（TBS系）で武井咲とW主演するタッキーこと滝沢秀明。タッキーが演じるのは妻帯者の副社長役で、中村蒼が演じる独身の宮沢綾と、武井を巡って恋のバトルを繰り広げる。 ファンならずとも期待値が高い作品だが、この新ドラマに関する情報が公開された直後から、タッキーファンが意外なことを心配しているという。「タッキーも今や34歳ですから妻帯者を演じるのはおかしくない年齢です。テレビ局側は、タッキーが妻帯者を演じることに意外性を感じるファンが多いだろうと予測していたのですが、ふたを開けてみるとタッキーの恋敵を演じる中村の演技力にタッキーが負けてしまうのでは、という点での心配の声があがっているというのです。ネット上では『ヤバイ！ 主役のタッキーが脇役の中村蒼に演技で食われる』『中村クン、タッキーに演技指導して』など、タッキーファンは中村の演技力を認めたうえでヤキモキしているようです」（女性誌編集者） 最近は時代劇出演が続いていたタッキー。久しぶりの現代モノで花を咲かせてもらいたいものだ。\n
1,11570186,サザエさん 作者が海岸を散歩していたから海産物の名前に,\n 戦後間もない1946年4月22日、福岡の地方紙『夕刊フクニチ』で『サザエさん』は始まった。高度経済成長や女性の社会進出など時代を反映しながら毎日、読者に笑顔を届けてきた『サザエさん』も、今年、生誕70年を迎える。 約6500話──1946年から1974年まで長谷川町子さん（享年72）は、『夕刊フクニチ』や『朝日新聞』で4コマ漫画『サザエさん』を描き続けた。 原作生誕70年を迎えた今年、長谷川さんの発掘イラストなどを集めた『長谷川町子の漫畫大會〜町子・戦中の仕事〜』（小学館）の出版など、記念イベントが続々と開かれている。\n『サザエさん』といえば、登場人物全員が海産物にちなんだ名前で知られている。長谷川町子美術館学芸員の相澤弘子さんがその理由を明かす。「町子は3人姉妹で、病気がちの妹の健康のためによく海岸を一緒に散歩していました。それで名前が全て海産物になったんです」 それから、長谷川さんは『サザエさん』に時代のトレンドを入れながら連載を続けた。「家のテーブルはちゃぶ台だったんですけど、ダイニングテーブルになりました。冷蔵庫も氷から電気になりました。ファッションも流行を追っています。1970年代になると公害問題も取り上げています。新聞連載の漫画だからこそ、時代を入れなければいけないと考えていたのだと思います」（相澤さん）※女性セブン2016年6月9・16日号\n
2,11532008,中居正広はお客さん待遇ではなかった！3回目の被災地訪問で見せた素顔とは？,\n SMAPの中居正広が三度目の被災地訪問を行い、自らフライパンを手に炊き出しに精を出した。 今回はメンバーの香取慎吾や「下町ロケット」の福澤克雄監督らを引き連れての訪問。3人の様子を伝える画像や動画がSNSに広まるなか、あるシーンが話題になっている。芸能ライターが語る。「調理中の中居が周りをキョロキョロしていると、仲間のスタッフらしき人物から『チャーハンかき回して！』と言わんばかりの指示を出され、慌ててフライパンを振るというシーンです。その指示に遠慮などは感じられず、スタッフ間の結束を感じさせました。この姿は中居が“お客さん待遇”で被災地訪問しているのではなく、一ボランティアとして訪れている証拠ですね」\n そんな中居のボランティア活動では、訪問場所もしっかり考えられているようだ。週刊誌の記者はこう指摘する。「最初に訪問したのは熊本市内の避難所で、2回目は南阿蘇村、そして3回目は700人以上が避難している御船町でした。訪問のたびに場所を変えているのに加え、今回の御船町はボランティアがあまり行き渡っていないと言われており、現地の人にとって中居の訪問は最高に嬉しいサプライズになったことでしょう」 普段はファンと一緒に写真を撮ることがないジャニーズタレントながら、満面の笑顔で記念写真に応じていた中居。その点、同行の香取はまだ笑顔に硬さも見られたが、今後も被災地訪問を続けていけばやがて、明るさ満点の慎吾スマイルを見せてくれるに違いないだろう。（白根麻子）\n
3,11579666,ベッキー絡みで炎上商法するしかない？ゲス極・川谷に続く”苦境”,\n 不倫報道で世間を騒がしたタレントのベッキー（32）が、今月13日に放送された『中居正広の金曜日のスマイルたちへ』（TBS系）に出演したのもつかの間、同騒動で一躍名前が広く知れ渡ったロックバンド「ゲスの極み乙女。」のボーカル川谷絵音（27）サイドには、またもや「ゲスい噂」が発覚し話題になっている。 ベッキーがテレビ出演を果たした数日前の5月9日に、ブログにて離婚を発表した川谷だが、その妻の妊娠中に浮気していたという疑惑だ。\n 川谷は5月22日に都内で開かれた野外フェスに出演。MCでは今回の騒動に直接触れることはなかったものの、「最近楽しいことがない」など意味深な発言を見せている。さらに前日にツイッターで「最近のどが原因不明でかなり良くない」と綴るなど、“かまってちゃんアピール”も甚だしい様子だ。こうした川谷の態度に、「無責任男な発言だよな」「自分が悲劇の主人公だと思ってるのでは？」「絶対不幸な自分に酔ってる」といった手厳しい声が続出している。■炎上商法成功も今後に不安 また今回の騒動に対して、川谷は当事者でありながら一番被害が少ないと指摘する声は多い。ベッキーは、レギュラー番組やCMといった全ての仕事が消滅。休業を余儀なくされたのに対し、川谷は一切おとがめなし。逆に不倫報道直後にリリースしたアルバム『両成敗』は7万枚以上を売り上げるなど、現在の音楽業界からするとかなり好調な記録を残している。 さらにツアーのチケットは先行予約の時点で完売状態。中には今回の騒動によって、「アーティストとして箔がついた」などの声も挙げる関係者もいるほどだ。結果として、川谷やバンドそのものを飛躍させることに一役買ってしまった部分は大きく、まさに炎上商法の成功とも言える。「騒動直後のベッキー・川谷両者に向けられたバッシングに対し、川谷が所属する事務所はテレビ関係者に“これ以上叩かないで”と懇願したそうです。しかし結局CDの売り上げは好調となり、炎上商法が成立した。そのため、今後は騒動を逆手にとった炎上商法を狙って行く可能性も否めません」（音楽関係者） しかし騒動発覚から半年間が経とうとしている現在、世間の川谷を見る目も大きく変わってきているのも現実だ。これだけ不倫騒動のイメージが定着したことで、メディアでへの露出や音楽活動に影響が出ることが懸念されている。テレビCMなどに川谷や...
4,11542071,登場から1カ月、新うたのおねえさんの評価は「超フツー」のガッカリ感,\n「おかあさんといっしょ」（NHK）の歌のおねえさんが小野あつこおねえさんに変わり、はや1カ月が過ぎた。そろそろ番組にも慣れる頃で新おねえさんの評価が聞こえてくるが、それがどうも芳しくないようなのだ。「よくないというより、評価しようがないというのが正しいようです。子供たちに聞いても母親に聞いても返ってくる答えは『なんか、フツー』ばかり。前任の三谷たくみおねえさんは個性的な顔で歌もべらぼうにうまかった。それに比べると新おねえさんは歌もルックスもしゃべりも平均点で、これといった特徴がないのでこのような評価になるのでしょう。ただ、はいだしょうこおねえさんもスプーの絵を描くなど個性を発揮し出したのは3年目になってから。あつこおねえさんもまだまだこれからではないでしょうか」（テレビ関係者）\n もっとも没個性なのはおねえさんだけではないとの指摘もある。現在のうたのおにいさんは横山だいすけおにいさんだが、歴代の田中星児、水木一郎、たいらいさおに比べれば普通だし、体操のおにいさんも先代の佐藤弘道おにいさんほどのインパクトは残せていない。「NHKはあえて特徴のない人を選んでいるのかもしれません。というのも先日、9代目うたのおにいさんを務めた杉田あきひろが薬物所持容疑で逮捕されました。このような事態を避けたいのでしょう。また、うたのおねえさん、おにいさんは行動を厳しく規制されます。恋愛、結婚、妊娠、海外旅行、車の運転、すべて禁止です。これを厳守できるような人間は、尖った所のない普通の人でないと無理でしょう。先代のたくみおねえさんが歴代1位タイとなる8年もの長い間務めたのは、後任探しに手間取ったからではないか」（テレビ関係者） 子供向けの番組にもかかわらず成人男性の視聴者も多い「おあかさんといっしょ」。あつこおねえさんには、そんなオヤジたちを満足されてくれるようなはっちゃけぶりを見せてもらいたいものだ。\n


In [30]:
data = pd.read_csv(summary_data_file_path)
data.head()

,article_id,text
0,11574091,連ドラ「せいせいするほど、愛してる」で、滝沢秀明が主演を務める
1,11574091,ファンは恋敵を演じる中村蒼の演技力に負けると心配していると女性誌編集者
2,11574091,ネットには「タッキーが脇役の中村蒼に演技で食われる」との声があるという
3,11570186,「サザエさん」の登場人物の名前が海産物にちなんでいる理由を紹介している
4,11570186,作者の長谷川町子さんが、姉妹と海岸をよく散歩していたことが理由だという
